<span STYLE="font-size:150%"> 
    Segment polygonal regions from microCT scans
</span>

Docker image: gnasello/slicer-env:2023-10-10 \
Latest update: 11 October 2023

- load image stack in Slicer
- segment mineralized tissue
- compute segmented statistics (volumes)

# Load libraries

In [ ]:
import pyslicer as ps
import slicer
from pathlib import Path
import pandas as pd
import trimesh
import vtk
import numpy as np

# Volume input

## Load `.nrrd` file into 3D Slicer

Write the path of the `.nrrd` file and load it to Slicer

In [ ]:
# this cell is tagged 'parameters'
volume_file = 'microCT_volume_preview/microCT_volume_preview.nrrd'
output_polygon_dir_path = 'segmented_volumes/polygonDefect'
output_cylinder_dir_path = 'segmented_volumes/originalDefect'
file_segmentation = 'segmented_volumes/Skull.seg.nrrd'
directory_notebook = Path().parent.absolute()
sample_name = directory_notebook.stem
camera_view_file = 'segmented_volumes/camera_view.csv'

Make ```segmented_volumes``` folder

In [ ]:
output_originalDefect_path = Path(output_cylinder_dir_path)

output_originalDefect_path.mkdir(parents=True, exist_ok=True)

In [ ]:
output_polygonDefect_path = Path(output_polygon_dir_path)

Get norebook directory name

In [ ]:
sample_name

# Visualize segmentations in 3D view

In [ ]:
# Switch to "One 3D view" layout
layoutManager = slicer.app.layoutManager()
layoutManager.setLayout(slicer.vtkMRMLLayoutNode.SlicerLayoutOneUp3DView)

# If you want to return later to the default layout (4-up), you can use:
# layoutManager.setLayout(slicer.vtkMRMLLayoutNode.SlicerLayoutFourUpView)

In [ ]:
ps.view.default_dark_3D_view()

## Get camera position in 3D view

Load `camera_view.csv` file

In [ ]:
df_camera = pd.read_csv(camera_view_file)
df_camera

In [ ]:
ps.view.set_camera_3Dview(position=df_camera['position'],
                          viewAngle=df_camera['viewAngle'][0],
                          viewUp=df_camera['viewUp'],
                          parallelScale=df_camera['parallelScale'][0])

# load `.vtk` file to 3D Slicer as model

In [ ]:
import slicer

# Path to your VTK file
vtk_file_path = str(Path(output_polygon_dir_path) / 'defectPolygon.vtk')

# Load model into the scene
model_node = slicer.util.loadModel(vtk_file_path)

if model_node:
    print("Model loaded successfully:", model_node.GetName())
else:
    print("Failed to load model.")


# Create minimal enclosing cylinder that is not restricted to axis alignment

In [ ]:
# Get polydata from the model node
polyData = model_node.GetPolyData()

# Make sure the mesh is triangulated
triFilter = vtk.vtkTriangleFilter()
triFilter.SetInputData(polyData)
triFilter.Update()
triPolyData = triFilter.GetOutput()

# Extract vertices
points = slicer.util.arrayFromModelPoints(model_node)

# Extract triangle faces
faces_vtk = triPolyData.GetPolys()
faces_vtk.InitTraversal()
idList = vtk.vtkIdList()

faces = []
while faces_vtk.GetNextCell(idList):
    if idList.GetNumberOfIds() == 3:
        faces.append([idList.GetId(0), idList.GetId(1), idList.GetId(2)])

faces = np.array(faces)

# Build Trimesh
mesh = trimesh.Trimesh(vertices=points, faces=faces, process=True)

# Compute minimum enclosing cylinder
cylinder_primitive = trimesh.bounds.minimum_cylinder(mesh)

# Get bounding_cylinder as mesh object
cylinder_mesh = mesh.bounding_cylinder

Print cylinder radius

In [ ]:
cylinder_primitive['radius']

In [ ]:
# Create a model in 3D Slicer with the bounding cylinder cylinder
enclosing_cylinder_model = ps.model.create_hollow_cylinder(height=cylinder_primitive['height'], 
                                                             radius_inner=0, radius_outer=cylinder_primitive['radius'], space =5, 
                                                             center=cylinder_mesh.center_mass,
                                                             direction=cylinder_mesh.direction.tolist(),
                                                             transform=False,
                                                             nameModel='EnclosingCylinder', 
                                                             color=(128/255, 174/255, 128/255), 
                                                             opacity=0.5)

# Create a cylinder covering the original defect area

In [ ]:
original_radius = max(cylinder_primitive['radius'], 2) #mm

defect_height = 0.8 #mm 

# Create a model in 3D Slicer with the bounding cylinder cylinder
original_defect_cylinder = ps.model.create_hollow_cylinder(height=defect_height, 
                                                           radius_inner=0, radius_outer=original_radius, space =5, 
                                                           center=cylinder_mesh.center_mass,
                                                           direction=cylinder_mesh.direction.tolist(),
                                                           transform=False,
                                                           nameModel='OriginalDefect', 
                                                           color=(216/255, 101/255, 79/255), 
                                                           opacity=0.5)

# Export cylinder and properties

In [ ]:
sample_name

Defect cylinder diameter

In [ ]:
# Scalar values
data = {'sample':sample_name,
        'original_defect_radius':original_radius}

df_polygon = pd.DataFrame(data, index=[0])

df_polygon.to_csv(output_originalDefect_path / (sample_name + '_original_defect.csv'), index = False)

df_polygon

Original Defect model

In [ ]:
model = original_defect_cylinder

filename_output = model.GetName() + '.vtk'

slicer.util.exportNode(model, output_originalDefect_path / filename_output)